In [39]:
import pandas as pd
import numpy as np
from pathlib import Path
import hvplot.pandas
import requests
import json

#import make_model_count as mm

# const variables
col_recall_number = "RecallCount"
col_date = "Date"
col_month = "Month"


def get_df():
    # get_df function load the csv file and creates the panda dataframe and return it
    # Furthermore, it also change the names of the columns in order to make these names are more readable
    vehicle_df = pd.read_csv("./resources/vrdb_60days_daily.csv")
    vehicle_df.head()
    vehicle_df = vehicle_df[["RECALL_NUMBER_NUM", "RECALL_DATE_DTE"]]
    vehicle_df.columns = [col_recall_number, col_date]
    vehicle_df = vehicle_df.groupby([col_date]).count()
    vehicle_df = vehicle_df.sort_values(by=col_date)
    vehicle_df = vehicle_df.reset_index()
    vehicle_df = vehicle_df[vehicle_df[col_date].str.contains("2022")]
    vehicle_df[col_month] = vehicle_df[col_date].astype(str).str[5:7]
    vehicle_df = vehicle_df.groupby([col_month]).sum()

    # since the vehicle recall count and the temperature has two different range of values
    #  it is more clearer if we add the pct_changes of the vehicle recalls to the chart
    vehicle_df = vehicle_df.pct_change()

    return vehicle_df


def recalls_temperature():
    #url = "https://api.weatherbit.io/v2.0/normals?lat=43.6532&lon=79.3832&start_day=01-01&end_day=12-31&tp=monthly&key=8eb2410b9bf74bb88acf2b1626fe05f5"
    #res = requests.get(url)
    # print(res.json())

    weather_data = open("./Resources/weather_data.json")
    weather_json = json.load(weather_data)
    arr = weather_json["data"]
    # weather_df = pd.read_json('''weather_json["data"]''')
    # df2 = pd.DataFrame.from_dict(weather_json, orient="index")

    min_temp = []
    max_temp = []
    months = []
    for wdata in arr:
        min = wdata["min_temp"]
        max = wdata["max_temp"]
        month = wdata["month"]
        min_temp.append(min)
        max_temp.append(max)
        months.append(month)

    weather_df = pd.DataFrame(
        {"Month": months, "MinTemp": min_temp, "MaxTemp": max_temp})
    weather_df = weather_df.pct_change()
    weather_df = weather_df.drop(columns=col_month)
    print(weather_df)

    comb_df = weather_df.copy()
    vehicle_df = get_df()
    comb_df[col_recall_number] = vehicle_df[col_recall_number].values

    return comb_df 


recalls_temperature()


# "RECALL_NUMBER_NUM",
# "YEAR",
# "MANUFACTURER_RECALL_NO_TXT",
# "CATEGORY_ETXT",
# "CATEGORY_FTXT",
# "MAKE_NAME_NM",
# "MODEL_NAME_NM",
# "UNIT_AFFECTED_NBR",
# "SYSTEM_TYPE_ETXT",
# "SYSTEM_TYPE_FTXT",
# "NOTIFICATION_TYPE_ETXT",
# "NOTIFICATION_TYPE_FTXT",
# "COMMENT_ETXT",
# "COMMENT_FTXT",
# "RECALL_DATE_DTE",
# "LAST_UPDATE_DATE",
# "MANUFACTURER_NAME_NM"


      MinTemp   MaxTemp
0         NaN       NaN
1   -0.337500  1.137931
2   -1.283019  1.290323
3    4.866667  0.605634
4    0.568182  0.232456
5    0.326087  0.163701
6    0.087432  0.061162
7   -0.085427 -0.031700
8   -0.285714 -0.157738
9   -0.530769 -0.286219
10  -0.967213 -0.455446
11 -27.000000 -0.600000


,MinTemp,MaxTemp,RecallCount
0,NaN,NaN,NaN
1,-0.337500,1.137931,0.666667
2,-1.283019,1.290323,4.200000
3,4.866667,0.605634,-0.576923
4,0.568182,0.232456,6.909091
5,0.326087,0.163701,0.252874
6,0.087432,0.061162,-0.917431
7,-0.085427,-0.031700,12.888889
8,-0.285714,-0.157738,-0.784000
9,-0.530769,-0.286219,9.851852
